In [135]:
import vk
import pandas as pd
import time

In [219]:
# opennig vk session with access token.
session = vk.Session(access_token='54d58d4edda257bd4aaec6527ae8183c2967a42a157568dc4ffcf966151991eaf26237b2de81400b4a7be')
api = vk.API(session)
# example use with id, my id
api.users.get(user_ids='architectofreality' )

[{'first_name': 'Alexander', 'last_name': 'Ovchinnikov', 'uid': 182927367}]

In [103]:
#searching for a keyword glaukoma starting 48 hours ago, max 200 counts can be scrapped at once
glaukoma=api.newsfeed.search(q='glaukoma',count=200, start_time=48)

### get last 200 posts --> take the time of the last post --> at the next iteration set the end time = time_last_post-1. Set the start time to 1 

In [104]:
%%time
glaukoma_rus=api.newsfeed.search(q='glaukoma',count=200, end_time=1512378120, start_time=1512378121-86400)

Wall time: 178 ms


In [105]:
# dropping the first element and creat a dataframe with the rest
df=pd.DataFrame(glaukoma_rus[1:])

In [106]:
df.head(2)

,attachment,attachments,comments,date,from_id,id,likes,marked_as_ads,owner_id,post_source,post_type,reposts,text
0,"{'type': 'photo', 'photo': {'pid': 456239772, ...","[{'type': 'photo', 'photo': {'pid': 456239772,...","{'count': 0, 'can_post': 1}",1512322597,-70740559,1034,"{'count': 0, 'user_likes': 0, 'can_like': 1, '...",0,-70740559,"{'type': 'api', 'platform': 'iphone'}",post,"{'count': 0, 'user_reposted': 0}",Наиболее частым заболеванием хрусталика являет...


In [107]:
df.columns

Index(['attachment', 'attachments', 'comments', 'date', 'from_id', 'id',
       'likes', 'marked_as_ads', 'owner_id', 'post_source', 'post_type',
       'reposts', 'text'],
      dtype='object')

In [108]:
#getting info of each user, deactivated column
ids=pd.DataFrame(api.users.get(user_ids=abs(df['owner_id'])))
ids.head()

,deactivated,first_name,last_name,uid
0,deleted,Elena,Zatylkina,70740559


In [231]:
%%time
desired_time = 86400*24 # how much back we want to go in seconds
date = time.time() # now
excess_time = True
all_df = pd.DataFrame()
c=1
while excess_time:
    print(c) # print iteration count
    g=api.newsfeed.search(q=['что лучше носить очки или линзы?'],count=200, end_time = date-1, start_time=1,extended=1)
    if len(g)>1: # check if g contains data
        all_df = pd.concat([all_df,pd.DataFrame(g[1:])])
        date=g[-1]['date']
    else:
        time.sleep(0.3)
    c+=1
    if date<time.time()-desired_time: # check termination condition
        excess_time = False

        # resetting the index it make cicles of 0-200
all_df.reset_index(inplace=True)

# set all negative ids to positive
all_df.owner_id=abs(all_df.owner_id) 

 # get all users
all_id=pd.DataFrame(api.users.get(user_ids=abs(all_df['owner_id'])))

# fill active users status with valid
all_id.fillna('Valid',inplace=True) 

# select all deactivated user ids
deactivated_ids = list(all_id[all_id['deactivated']!='Valid'].uid)

# select all active user posts
active_users = all_df[-all_df.owner_id.isin(deactivated_ids)] 

1
2
3
Wall time: 14.9 s


In [244]:
active_users.group.iloc[0]

{'gid': 36620432,
 'is_admin': 0,
 'is_closed': 0,
 'is_member': 0,
 'name': 'ОПТИК-ШОП- контактные линзы доставка Тюмень!',
 'photo': 'https://pp.userapi.com/c840124/v840124562/38669/l9uDJ85E6Kw.jpg',
 'photo_big': 'https://pp.userapi.com/c840124/v840124562/38667/d6MTeWy5sOA.jpg',
 'photo_medium': 'https://pp.userapi.com/c840124/v840124562/38668/JiSlADNQvxA.jpg',
 'screen_name': 'opticianshop',
 'type': 'group'}

In [230]:
#active_users.iloc[7]['text']